In [13]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [17]:
def get_stock_data(ticker):
    stock = yf.download(ticker)
    return stock.dropna()

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
stock_data = {ticker: get_stock_data(ticker) for ticker in tickers}


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [18]:
stock_data

{'AAPL':                   Open        High         Low       Close   Adj Close  \
 Date                                                                     
 1980-12-12    0.128348    0.128906    0.128348    0.128348    0.099450   
 1980-12-15    0.122210    0.122210    0.121652    0.121652    0.094261   
 1980-12-16    0.113281    0.113281    0.112723    0.112723    0.087343   
 1980-12-17    0.115513    0.116071    0.115513    0.115513    0.089504   
 1980-12-18    0.118862    0.119420    0.118862    0.118862    0.092099   
 ...                ...         ...         ...         ...         ...   
 2023-09-01  189.490005  189.919998  188.279999  189.460007  189.460007   
 2023-09-05  188.279999  189.979996  187.610001  189.699997  189.699997   
 2023-09-06  188.399994  188.850006  181.470001  182.910004  182.910004   
 2023-09-07  175.179993  178.210007  173.539993  177.559998  177.559998   
 2023-09-08  178.350006  180.240005  177.789993  178.179993  178.179993   
 
               

In [19]:

# Combine data of all tickers into one DataFrame
combined_data = pd.concat(stock_data, axis=1)

# Split the data into training and testing datasets (80% training, 20% testing)
train_data, test_data = train_test_split(combined_data, test_size=0.2, shuffle=False)

# Scale the data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled = scaler.transform(test_data)


In [20]:
X_train = []
y_train = []

for i in range(60, len(train_scaled)):
    X_train.append(train_scaled[i-60:i])
    y_train.append(train_scaled[i])

X_train, y_train = np.array(X_train), np.array(y_train)

X_test = []
y_test = []

for i in range(60, len(test_scaled)):
    X_test.append(test_scaled[i-60:i])
    y_test.append(test_scaled[i])

X_test, y_test = np.array(X_test), np.array(y_test)


In [21]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=X_train.shape[2]))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32)


Epoch 1/50


2023-09-10 21:21:50.284767: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:21:50.642441: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:21:51.011025: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:21:51.259756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:21:51.528850: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


268/268 [==============================] - 9s 25ms/step - loss: nan
Epoch 2/50
268/268 [==============================] - 6s 23ms/step - loss: nan
Epoch 3/50
268/268 [==============================] - 6s 23ms/step - loss: nan
Epoch 4/50
268/268 [==============================] - 6s 23ms/step - loss: nan
Epoch 5/50
268/268 [==============================] - 6s 23ms/step - loss: nan
Epoch 6/50
268/268 [==============================] - 6s 23ms/step - loss: nan
Epoch 7/50
268/268 [==============================] - 6s 23ms/step - loss: nan
Epoch 8/50
268/268 [==============================] - 6s 23ms/step - loss: nan
Epoch 9/50
268/268 [==============================] - 6s 23ms/step - loss: nan
Epoch 10/50
268/268 [==============================] - 6s 23ms/step - loss: nan
Epoch 11/50
268/268 [==============================] - 6s 23ms/step - loss: nan
Epoch 12/50
268/268 [==============================] - 6s 23ms/step - loss: nan
Epoch 13/50
268/268 [==============================] - 6s 24

In [24]:
predictions = model.predict(X_test)
predictions


66/66 [==============================] - 1s 9ms/step


array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [23]:
mse = np.mean(np.square(y_test - predictions))
print("Mean Squared Error:", mse)


Mean Squared Error: nan


In [25]:
# Assuming the predictions are for the closing prices
predicted_closes = predictions[:, tickers.index('AAPL')]

# Calculate the returns of the predicted prices
predicted_returns = np.diff(predicted_closes) / predicted_closes[:-1]

# Calculate the covariance matrix of the predicted returns
cov_matrix = np.cov(predicted_returns)

# Portfolio optimization
from scipy.optimize import minimize

def objective(weights):
    return -np.dot(predicted_returns, weights)

constraints = (
    {'type': 'eq', 'fun': lambda weights: np.dot(weights, np.ones(len(weights)))},
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
)

bounds = [(-1, 1) for stock in range(len(tickers))]

initial_weights = [1./len(tickers) for stock in tickers]

solution = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

print("Optimal weights:", solution.x)


ValueError: shapes (2094,) and (4,) not aligned: 2094 (dim 0) != 4 (dim 0)

In [7]:
scalers = {}
for ticker in tickers:
    scalers[ticker] = MinMaxScaler()
    stock_data[ticker] = scalers[ticker].fit_transform(stock_data[ticker])


In [8]:
def prepare_data(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

time_steps = 10
X_train, y_train = {}, {}
for ticker in tickers:
    X_train[ticker], y_train[ticker] = prepare_data(stock_data[ticker], time_steps)


In [9]:
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=50))
    model.add(Dense(units=6))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

models = {}
for ticker in tickers:
    models[ticker] = create_model((X_train[ticker].shape[1], X_train[ticker].shape[2]))


2023-09-10 21:05:27.508651: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-09-10 21:05:27.508693: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-09-10 21:05:27.508708: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-09-10 21:05:27.509108: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-10 21:05:27.509477: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
epochs = 50
for ticker in tickers:
    print(f"Training {ticker} Model")
    models[ticker].fit(X_train[ticker], y_train[ticker], epochs=epochs, batch_size=32)


Training AAPL Model
Epoch 1/50


2023-09-10 21:05:39.320200: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:05:39.743937: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:05:40.365452: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:05:41.837182: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:05:42.912661: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


337/337 [==============================] - 12s 21ms/step - loss: 8.7511e-04
Epoch 2/50
337/337 [==============================] - 6s 19ms/step - loss: 2.3887e-04
Epoch 3/50
337/337 [==============================] - 6s 18ms/step - loss: 2.3306e-04
Epoch 4/50
337/337 [==============================] - 6s 19ms/step - loss: 2.1962e-04
Epoch 5/50
337/337 [==============================] - 6s 18ms/step - loss: 1.9935e-04
Epoch 6/50
337/337 [==============================] - 6s 18ms/step - loss: 1.9915e-04
Epoch 7/50
337/337 [==============================] - 6s 18ms/step - loss: 1.9549e-04
Epoch 8/50
337/337 [==============================] - 6s 18ms/step - loss: 1.8317e-04
Epoch 9/50
337/337 [==============================] - 6s 18ms/step - loss: 1.7786e-04
Epoch 10/50
337/337 [==============================] - 6s 18ms/step - loss: 1.8299e-04
Epoch 11/50
337/337 [==============================] - 6s 18ms/step - loss: 1.5714e-04
Epoch 12/50
337/337 [==============================] - 6s 18ms

2023-09-10 21:10:58.815484: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:10:59.150128: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:10:59.242269: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:10:59.418941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 21:10:59.569326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


295/295 [==============================] - 8s 19ms/step - loss: 0.0024
Epoch 2/50
295/295 [==============================] - 5s 19ms/step - loss: 2.6202e-04
Epoch 3/50
295/295 [==============================] - 5s 18ms/step - loss: 2.5234e-04
Epoch 4/50
295/295 [==============================] - 5s 19ms/step - loss: 2.2845e-04
Epoch 5/50
295/295 [==============================] - 5s 18ms/step - loss: 2.1853e-04
Epoch 6/50
295/295 [==============================] - 6s 19ms/step - loss: 2.1608e-04
Epoch 7/50
 66/295 [=====>........................] - ETA: 4s - loss: 2.4296e-04

KeyboardInterrupt: 